*Properly used, positive reinforcement is extremely powerful. - [B. F. Skinner](https://www.brainyquote.com/authors/b-f-skinner-quotes)*

*once one of my pups found half a roast chicken in the corner of a parking lot and we had to visit that exact same corner every day for about fifty years because for dogs hope springs eternal when it comes to half a roast chicken - [darth](https://twitter.com/darth/status/1057075608063139840)* (possibly embed tweet)

Tic-Tac-Toe is a simple game. If both sides play perfectly, neither can win. But if one plays imperfectly, the other can exploit the flaws in the other's strategy. 

Does that sound a little like trading?

In this post, we will explore reinforcement learning, and apply it, first to learn an algorithm to play Tic-Tac-Toe, and then learn to trade a moderately non-random price series.

### Tic-Tac-Toe With Simple Reinforcement Learning

Here's an algorithm that will learn an exploitive Tic-Tac-Toe strategy, and adapt over time if its opponent learns:

1) Make a big table of all possible Tic-Tac-Toe boards. 

2) Initialize the table to assign a value of 0 to each board, 1.0 where X has won, -1.0 where O has won.

3) Play with your opponent. At each move, pick the best available move in your table, or if several are tied for best, pick one at random. Occasionally, pick a move at random just to make sure you explore the whole state space, and to keep your opponent on their toes. 

4) After each game, back up through all the boards that were played. Update the value table as follows:
	- When X wins, update each board's value part of the way to 1. 
	- When O wins, update part of the way to -1.  
	- When they tie, update part of the way to 0 

This is a profoundly dumb algorithm in the finest sense. It knows almost nothing about the game of Tic-Tac-Toe and yet it works. It can't reason about the game. It needs a lot of training. It can't generalize to boards it hasn't seen (footnote: even if they are isomorphic to boards it has seen. When you think about it, there are only 3 starting moves, board center, corner, center side. Flipping or rotating the board shouldn't change the value of a position or how to play it.) . It doesn't learn the globally optimal strategy. 

But over time, this algorithm will learn, it will exploit flaws in its opponent's strategy, and if the opponent changes tactics, over time it will adapt.

This is *reinforcement learning*. (link to code)

More sophisticated reinforcement learning algorithms enable [robots to walk on four or two legs](https://www.youtube.com/watch?v=xXrDnq1RPzQ), [driverless cars to drive](https://www.youtube.com/watch?v=eRwTbRtnT1I), computers to play [Atari](https://deepsense.ai/playing-atari-with-deep-reinforcement-learning-deepsense-ais-approach/) and [poker](https://www.engadget.com/2017/02/10/libratus-ai-poker-winner/?guccounter=1) and [Go](https://deepmind.com/blog/article/alphago-zero-starting-scratch), in some cases better than humans. 

In the parts that follow, we'll extend the Tic-Tac-Toe example to more complex deep reinforcement learning, and try to build a reinforcement learning trading robot.

### Reinforcement Learning Concepts

But first, how does reinforcement learning in general work?

![agent-environment](RL1.png "Figure 1")

all these figures are from Silver - http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html

1) At time *t*, The *agent* observes the environment *state* *s<sub>t</sub>* (the Tic-Tac-Toe board).

2) From the set of available actions (the open squares), the agent takes *action* *a<sub>t</sub>* (the move giving the highest-scoring board). 

3) The environment updates at the next *timestep* *t+1* to a new state *s<sub>t+1</sub>*. In Tic-Tac-Toe this is the board resulting from the opponent's move. In a complex environment like a car on a road, the new state may be partly determined by the agent's actions (you turned left and accelerated) and partly by visible or hidden complexities in the environment (a dog runs into the road). And the new state may not be deterministic, it may be stochastic, where some things occur randomly with probabilities dependent on the visible and hidden state and the actions of the agent.

4) The environment generates a *reward*. In Tic-Tac-Toe you get a reward when you win, lose, or draw. In Space Invaders, you win points at various times when you hit different targets. When training a self-driving car, machine learning engineers have to design rewards for staying on the road, getting to the destination, including negative rewards for e.g. collisions.

The technical name for this environment is a [Markov Decision Process](https://en.wikipedia.org/wiki/Markov_decision_process) (MDP). Reinforcement learning always has states, actions, transitions between states, rewards, and an agent that chooses actions, using a cycle of observing the state, acting, getting a reward and repeating with a new state. 


### Reinforcement Learning Variations

The agent always has a *policy function*, which chooses the best action based on the environment state. It *may* have the following components:

- *Model* - An internal representation of the environment. Our agent stores the board and it knows some state-action pairs result in the same state as other state-action pairs. Therefore it can be considered *model-based* (although it doesn't model the full MDP with all transition probabilities). Other agents are *model-free*. They choose actions without explicitly storing an internal model of the state or modeling state transitions. The understanding of the environment is implicit in the policy function (footnote. for instance if instead of a table with all possible Tic-Tac_Toe boards we used a table mapping (boards, action) pairs to values. Then we wouldn't be modeling internally what happens after a move, i.e. several (board, action) pairs arrive at the same board. We would just evaluate state, action pairs directly without an internal model. That would work too, just take longer to train since it would be a bigger table.).

- *State value function* - A way to score a state (our V table).

- *State-action value function* - A way to score the value of an action in a given state, i.e. a state-action pair, commonly termed a *Q-value function*.

Just as there are many algorithms for regression or classification, there are many variations in reinforcement learning architectures. New approaches are created constantly as the field advances. Based on which components a reinforcement learning algorithm uses to generate the workflow shown in Figure 1, it can be characterized as belonging to different flavors or reinforcement learning.

![taxonomy](RL3.png "Figure 2")

All reinforcement learning variations learn using a similar workflow:

1) Initialize the algorithm with a naive, possibly random policy.

2) Using the policy, take actions, observe states before and after actions, experience rewards.

3) Fit a model which improves the policy.

4) Go to 2) and iterate, collecting more experience with the improved policy, and continuing to improve it.

(find/make a flowchart)

As we continue to iterate, we improve the algorithm.

### Reinforcement Learning In Context

In a [previous post](https://alphaarchitect.com/2017/09/27/machine-learning-investors-primer/) we discussed types of machine learning:

Supervised learning: Any algorithm that predicts labeled data. Regression predicts a continuous response variable (next quarter's real GDP growth, next month's stock return).  Classification predicts a categorical response variable (recession or recovery, next month's return quintile). 

Unsupervised learning: Any algorithm that summarizes or learns about unlabeled data, such as clustering or dimensionality reduction. 

Every data set is either labeled or unlabeled, right? Between supervised and unsupervised, that must cover everything, right? It's like those two books, [What They Teach You At Harvard Business School](https://www.amazon.com/What-Teach-Harvard-Business-School/dp/0141037865) and [What They Don't Teach You At Harvard Business School](https://www.amazon.com/What-Teach-Harvard-Business-School/dp/0553345834), they must cover all human knowledge. 

Nevertheless reinforcement learning is considered the third major machine learning paradigm. Consider the tic-tac-toe robot:

- The agent doesn't have fixed training data, it discovers data via an unsupervised process and learns a policy.

- The rewards can be viewed as labels generated by a supervisor. But rewards aren't directly related to any specific prediction or action. If the agent shoots a target in Space Invaders, it has to figure out which action or actions possibly many timesteps earlier contributed to the reward (the credit assigment problem). 

- The agent's interactions with the environment *shape* that environment and generate a *feedback loop*. A Space Invaders agent changes the world by shooting targets; a self-driving car doesn't modify the road, but its presence and behavior modify how other vehicles behave, and what environment the algorithm encounters.

- In supervised learning, the algorithm optimizes model parameters over training data to minimize an error, like mean squared error or cross-entropy. In reinforcement learning, the algorithm optimizes the model parameters over the state space it encounters, and maximizes the expected cumulative reward generated by the Markov Decision Process (MDP) over time.

I view reinforcement learning as meta-supervised-learning. It's the application of supervised machine learning to [*optimal control*](https://en.wikipedia.org/wiki/Optimal_control). We learn behavior policies to maximize reward in a complex dynamic environment which may also have random and hidden elements.

Many disciplines have encountered these problems in various forms and developed methodologies to solve them: 

- Business/Operations Research: dynamic pricing of airline seats or other products to maximize profits under changing inventory, production, demand conditions
- Economics: optimal Fed interest rate policy to maximize growth in a dynamic economy
- Engineering: auto-pilots, spacecraft navigation, complex robots and industrial automation
- Psychology: Stimulus-response, positive and negative reinforcement
- Neuroscience: The brain's chemical reward loop, how children learn to walk and talk, or catch a ball
- Mathematics: Control theory, game theory, optimization

In a trading context, optimal control is how we use a market signal to create trading strategy. 
- The signal can be regression, predicting a continuous variable (link) or classification, predicting a discrete variable e.g. outperform/underperform (binary classification) or quintiles (multinomial classification) (link). 
- The reward can be raw return or risk-adjusted return (Sharpe). 
- Reinforcement learning  can be used to find an optimal policy (trading strategy) to maximize the desired reward.

![connections](RL2.png "Figure 2")



### Deep Reinforcement Learning

How do we get from our simple Tic-Tac-Toe algorithm to an algorithm that can drive a car or trade a stock?

We can view our table lookup-based algorithm as a model with a *linear value function approximator*. If we represent  our board as a one-hot feature vector based on which known board it represents, and view the lookup table as a vector, then if we dot-multiply the one-hot feature vector by the lookup table values, we get a linear value function.

Our linear value function approximator takes a board, converts it to a feature vector and outputs a linear function of that feature vector to generate a value for that board. We can swap that linear function for a nonlinear function, such as neural network. When we do that, we get our first, very crude, deep reinforcement learning algorithm.

Our new algorithm is:

1) Initialize our neural network to random weights

2) Play a game with our opponent

3) At the end of the game, append each board we encountered into a nx9 array (our predictors) associated with the outcome of the game (our response)

4) Fit the neural network to the recent predictors and responses we've seen (run one or more iterations of stochastic gradient descent)

5) Go to 2) and gather more data.

This will work, although it takes a long time to train and makes our initial brute force method even more inefficient. (see code). 

But in a nutshell, that is how a self-driving car could work. 

- The state is represented by a giant array of inputs from all the onboard cameras and sensors.

- The actions are: turn the steering wheel, accelerate, and brake.

- Positive rewards come from staying on the road and arriving safely to the destination, and negative rewards from breaking traffic rules or colliding.

- The real world provides the state transitions.

- And we train a complex neural network to do all the right things involved in detecting and interpreting all the objects in the environment and navigating from point A to point B.

Table lookup can't scale to high dimensional or continuous action or state spaces. And a linear function approximator can't learn nonlinear behavior. With deep neural networks, reinforcement learning algorithms can learn complex emergent behavior. 



Finally, a trading example.

- inspired by Gordon Ritter paper https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3015609

- let's create a stocks. give them each a random walk, so they are flat. now make it springy. random walk plus momentum, plus attraction back to the mean, , plus trend.

- let reinforcement learning trade it.



Conclusions

Similar paradigm to Andrew Lo's adaptive markets https://alo.mit.edu/book/adaptive-markets/

bridges gap between prediction and trading. sometimes you find a very small predictive R-squared can lead to large gains. hypothetically, suppose you have a stock market that yields a 5% return. and the top 1% of days are up 5% and the top bottom percent of days are down 1%. Suppose your prediction model gets all of those extreme days right. With 51% accuracy you doubled your return. Suppose you get 51% evenly distributed, you get a tiny increment.

do a lot of prediction, attempt to use it as input to a strategy. we find very often in deep learning that when we train complex models with many different moving parts end to end, this results in interesting emergent behavior. makes sense to try to train for prediction and control end to end.

algorithms own very short term hf trading. with algorithms like RL they can move up the timescale.

JPMorgan and others reportedly using RL to trade in the real world https://medium.com/@ranko.mosic/reinforcement-learning-based-trading-application-at-jp-morgan-chase-f829b8ec54f2 - https://arxiv.org/pdf/1802.03042.pdf

Deep reinforcement learning is hard. We're not yet at the point where you can point a universal agent at a problem and expect a quick solution. Deep rl is very data-hungry or sample-inefficient. It's not something you can do with a few years of quarterly financials.

You also need a lot of experimentation and tuning and testing in an adversarial environment. 

https://www.alexirpan.com/2018/02/14/rl-hard.html

rl is not infallible or perfectly stable. reinforcement learning algorithms that constantly train on recent experience forget things they learned at the beginning of training, and suddenly start to underperform. you need good tradeoff between exploitation and exploration that covers the full state/action space. Commercial pilots can go through thousands of flights without encountering a real in-flight emergency. In order to handle those emergencies, they need recurrent training where all kinds of failures are simulated. 

Algorithms can be exploited. Self-driving vehicles on the streets of New York or New Delhi seem unlikely in the near future, without changes like protected lanes for self-driving vehicles and strong enforcement. If pedestrians know that the other driver is always going to stop for them no matter what, they will learn to just cross at the red light, never mind traffic. They can even wear a stop sign on a T-shirt. It's not a matter of how good the self-driving technology is, it's a matter of game theory. Knowing that the other driver is a fallible human who at best may be angry and honk and give you the finger, and at worst may be on a cell phone and not even see you tends to concentrate the mind. 

In our tic-tac-toe example, the reinforcement learning algorithm adapts to the player. The player who understands the algorithm can exploit this, taking a loss to teach the algorithm a bad example the human player can exploit many times before the algorithm catches up. It's like a poker player showing a bluff to get other players to loosen up, or the nut flush to get a little more respect for bluffs. 

In a worst case, an adversarial sticker can make image recognition think a banana is a toaster or an adversarial temporary tattoo can defeat face recognition 
https://medium.com/deep-learning-cafe/neural-networks-easily-fooled-e19bf575b527
https://cvdazzle.com/

This is a problem for trading with reinforcement learning. if a market maker algorithm trades on patterns, adversarial algorithms can learn to exploit it. you can't 

nevertheless anecdotally poker bots taking over,

there's a need to combine brute force RL with some reasoning about the world or the game, which AlphaGo and AlphaZero are able to do to some extent.


like the man in the moliere play, you've been doing optimal control your whole life without realizing it.

Further reading

- [UCL course by David Silver (videos and lecture notes)](http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html)
- [Stanford course](http://web.stanford.edu/class/cs234/schedule.html)
- [Berkeley course](http://rail.eecs.berkeley.edu/deeprlcourse/)
- [Reinforcement Learning: An Introduction, Richard S. Sutton and Andrew G. Barto](http://incompleteideas.net/book/the-book-2nd.html)
- [Algorithms for Reinforcement Learning, Csaba Szepesvári](https://sites.ualberta.ca/~szepesva/papers/RLAlgsInMDPs.pdf)
- http://karpathy.github.io/2016/05/31/rl/
- https://arxiv.org/pdf/1802.03042.pdf


